In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-26"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10685,2024-02-26,Eua Nba,22:00,Memphis Grizzlies,Brooklyn Nets,1.93,1.93,213.5,1.81,2.00,1.5,1.94,1.74,ImLcQ9Ap,0.518135,0.518135,0.552486,0.500000,0.036269,0.4,0.6,NaN,NaN,354.486,164.830951,0.464986,323.988,178.276805,0.550257,379.520,401.362,622.63,196.23,1.0,0.0,0.0,0.0,0.000000,0.070525,0.076859,3.04,0.608,1.529605,0.326723,0.4,0.073277,0.67,0.134,6.940299,0.361349,0.2,-0.161349
10686,2024-02-26,Ásia Copa Asiática,12:00,Emirados Árabes Unidos,Siria,2.18,1.64,147.5,1.86,1.80,1.5,2.01,1.50,EBjVlpw5,0.458716,0.609756,0.537634,0.555556,0.068472,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.199915,0.023184,0.205484,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10687,2024-02-26,Eua Ncaa,21:30,Bethune Cookman,Alabama State,1.67,2.28,139.5,1.91,1.83,-4.5,2.05,3.20,M1Hqgcwk,0.598802,0.438596,0.523560,0.546448,0.037399,0.8,0.8,NaN,NaN,142.282,19.868866,0.139644,108.890,38.221714,0.351012,167.744,139.364,155.21,100.75,0.0,0.0,0.0,0.0,0.218398,0.030251,0.309780,3.80,0.760,0.881579,0.540722,0.7,0.159278,-2.78,-0.556,-2.302158,0.405729,0.3,-0.105729
10688,2024-02-26,Eua Ncaa,22:30,Arkansas-Pine Bluff,Texas Southern,2.10,1.73,146.5,1.80,1.91,1.5,1.95,1.63,pOtGOXJT,0.476190,0.578035,0.555556,0.523560,0.054225,0.2,0.6,NaN,NaN,145.746,49.239616,0.337845,130.594,26.219683,0.200772,125.424,148.990,129.60,154.05,0.0,0.0,0.0,0.0,0.136621,0.041931,0.126410,0.01,0.002,550.000000,0.000000,0.0,0.000000,0.43,0.086,8.488372,0.464607,0.4,-0.064607
10689,2024-02-26,França Copa Da França,14:30,Chalons-Reims,Vichy,2.04,1.73,160.5,1.87,1.87,1.5,1.89,1.60,0OnIR0VH,0.490196,0.578035,0.534759,0.534759,0.068231,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,148.05,0.00,0.0,0.0,0.0,0.0,0.116288,0.000000,0.117513,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10690,2024-02-26,Mundo Americup,23:10,México,República Dominicana,1.75,1.98,154.5,1.83,1.87,-1.5,1.87,1.96,ruWV8Jft,0.571429,0.505051,0.546448,0.534759,0.076479,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.087204,0.015289,0.033232,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:30,Eua Ncaa,Bethune Cookman,Alabama State,139.5,1.91,1.0,Over
